In [1]:
%matplotlib inline
import numpy as np

from business_bayes import bayesian_search

## Bayesian Search Theory

* [The problem](#The-problem)
* [A concrete example](#A-concrete-example)
    * [Problem specification](#Problem-specification)
    * [Observing the result of one search](#Observing-the-result-of-one-search)
    * [Choosing the next box to search](#Choosing-the-next-box-to-search)

### The problem

* The search space has been divided into $n$ *disjoint* boxes: the object is located in one (and only one) of these
* We have a prior belief of what the probabilities are of finding the object in each box
* For each box, we know the probability of finding the object after searching the box *given the object is actually in that box*
* For each box, we know how much performing a search of that box costs

Given all that, **how should we choose the sequence of boxes to search to minimize the costs of finding the object?**

### A concrete example


#### Problem specification

##### the search space

So our assumption is there's three possible causes for the slump in sales; we assume only one of these is happening.

1. a bug in the backend code
2. a bug in the frontend code
3. something else

##### the prior probability

We don't have good reasons to believe any one of these is more likely than any of the others, so we'll start by assigning equal probabilities to the three options:

* $P(backend) = \frac{1}{3}$
* $P(fronted) = \frac{1}{3}$
* $P(other) = \frac{1}{3}$

In [2]:
p0 = np.ones(3) / 3.

##### the probability of finding the object in a box given the object is actually there

As stated, the only thing we *can* do is look over the code. We're more experienced as a backend developer, so we'll rate our chances of finding a backend bug by visual inspection to be higher than finding a frontend bug by visual inspection. We do not know how to search for "something else", so we rate our chances of finding the cause there by inspection to be 0:

* $P(found \mid backend) = 0.4$
* $P(found \mid fronted) = 0.1$
* $P(found \mid other) = 0.0$

In [3]:
p_found_given_box = np.array([0.4, 0.1, 0])

##### the cost of searching each box

Let's say we work in chunks of 1 hour: the cost of looking over the frontend code and the backend code in that case is the same, 1. We might as well say that the cost of blue-sky thinking about other possible causes for an hour is also 1, but as we said above the probability of that being useful is 0.

* $cost\_searching\_backend = 1.$
* $cost\_searching\_frontend = 1.$
* $cost\_searching\_other = 1.$

In [4]:
costs = np.ones(3)

#### Observing the result of one search

How do our initial probabilities $p_0$ change after, say, poring over the backend code for an hour and **not** having found the bug? 

Intuitively, we know we now have a somewhat lower belief in there being a backend bug, and a somewhat higher belief in the bug being caused by the frontend or by something else.

If the assumptions we've made until now hold, we can use Bayes to exactly quantify **how much** these beliefs should change, let's do the math in the concrete case of having searched for an hour in the backend code and not having found anything.

```
P(A|B) = P(B|A)P(A) / P(B)

P(backend_found | backend) = 0.6
P(backend_not_found | backend) = 1 - P(backend_found | backend) = 0.6

P(backend_not_found | backend) = 0.6
P(backend_not_found | frontend) = 1
P(backend_not_found | other) = 1

P(backend_not_found) = p(backend_not_found | backend) * P(backend) + P(frontend) + P(other)
```

So the posterior probabilities, after searching in the backend code for 1 hour and not finding anything:

```
P(backend | backend_not_found) = P(backend_not_found | backend) * P(backend) / P(backend_not_found)
P(frontend | backend_not_found) = P(frontend) / P(backend_not_found)
P(other | backend_not_found) = P(other) / P(backend_not_found)
```

The bayesian update is implemented by the `bayesian_search.observe` function:

In [6]:
# we've searched in the backend code, which is the box with index 0:
search0 = 0

# get updated probabilities after taking this information into account
p1 = bayesian_search.observe(p0, search0, p_found_given_box[search0])
print(p1)

[0.23076923 0.38461538 0.38461538]


This result is a quantified version of our intuition: we're now less confident that there's a backend problem.

#### Choosing the next box to search